In [1]:
import pandas as pd
from pandas_datareader import data as pdr
import numpy as np 
import datetime 
import matplotlib.pyplot as plt  
import plotly.graph_objs as go  
%matplotlib inline

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot


print(__version__)


4.14.3


In [2]:
# YoY는 Year on Year의 약어인데, 전년 동기 대비 증감율
# QoQ는 Quarter on Quarter의 약어인데, 직전 분기 대비 증감율
# YTD는 Year To Date의 약어인데, 연초 누계, 연초 누계 대비 증감율 혹은 연초 대비 증감율
#   YTD의 첫번째 사용예는 단순히 올해 초부터 현재까지 누적값
#   YTD의 두번째 사용예는 "올해 현재까지의 누적값"과 "올해 현재 날짜와 같은 전년 날짜까지의 누적값"
#   YTD의 세번째 사용예는 올해 첫날부터 현재까지의 수익률
# %와 %p(point)
    # %는 증감율 혹은 비율을 표시할 때 사용 
    # %p(point)는 단위가 %인 비율의 증감을 의미

# df = pdr.get_data_yahoo('005930.KS')#삼성전자
# df = pdr.get_data_yahoo('066570.KS')#LG전자
# df = pdr.get_data_yahoo('005380.KS')#현대차
# df = pdr.get_data_yahoo('068270.KS')# 셀트리온 
# df = pdr.get_data_yahoo('035420.KS')# NAVER 
# df = pdr.get_data_yahoo('010060.KS')# OCI 
df = pdr.get_data_yahoo('068760.KQ')# OCI 
df

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2016-01-20,17180.199219,16106.400391,16993.500000,16293.200195,254722.0,16293.200195
2016-01-21,16620.000000,16013.099609,16433.199219,16293.200195,156843.0,16293.200195
2016-01-22,16433.199219,15499.500000,16386.599609,15966.400391,175944.0,15966.400391
2016-01-25,16013.099609,15639.599609,16013.099609,15873.000000,90294.0,15873.000000
2016-01-26,16153.099609,15639.599609,15779.599609,15733.000000,77758.0,15733.000000
...,...,...,...,...,...,...
2018-01-26,99900.000000,94800.000000,96000.000000,97400.000000,890629.0,97400.000000
2018-01-29,101900.000000,94400.000000,94700.000000,99900.000000,1145142.0,99900.000000
2018-01-30,103800.000000,97900.000000,101200.000000,99500.000000,1109788.0,99500.000000


In [127]:
# 미국 주식 시장의 SP 500의 기간에대한 모든 ticker 데이터 가져오기
# Modify these date ranges each week.
# 주별로 수정해야 한다. 
# 특정일 기준으로 기간 데이터를 가져온다
start_sp = datetime.datetime(2010, 1, 1)
end_sp = datetime.datetime(2021, 1, 18)

# This variable is used for YTD performance.
# YTD 올해 첫날부터 현재까지의 수익률
end_of_last_year = datetime.datetime(2020, 12, 31)

# These are separate if for some reason want different date range than SP.
stocks_start = datetime.datetime(2021, 1, 11)
stocks_end = datetime.datetime(2021, 1, 15)

def get(tickers, startdate, enddate):
  def data(ticker):
    return (pdr.get_data_yahoo(ticker, start=startdate, end=enddate))
  datas = map (data, tickers)
  return(pd.concat(datas, keys=tickers, names=['Ticker', 'Date']))



In [222]:
kospi = pdr.get_data_yahoo('KOSPI',start_sp, end_sp)
kospi
# sp500 = pdr.get_data_yahoo('^GSPC',start_sp, end_sp)
# sp500

RemoteDataError: No data fetched for symbol KOSPI using YahooDailyReader

In [129]:
# 포트폴리오의 구성
tickers = ['AAPL', 'JNJ', 'MCD','MTCH','NFLX','GOOG', 'WMT','FB','TWTR' ]
all_data = get(tickers, start_sp, end_sp)

all_data

High        Low       Open      Close       Volume  \
Ticker Date                                                                  
AAPL   2009-12-31   7.619643   7.520000   7.611786   7.526071  352410800.0   
       2010-01-04   7.660714   7.585000   7.622500   7.643214  493729600.0   
       2010-01-05   7.699643   7.616071   7.664286   7.656429  601904800.0   
       2010-01-06   7.686786   7.526786   7.656429   7.534643  552160000.0   
       2010-01-07   7.571429   7.466071   7.562500   7.520714  477131200.0   
...                      ...        ...        ...        ...          ...   
TWTR   2021-01-11  49.490002  45.150002  47.090000  48.180000   61647600.0   
       2021-01-12  49.130001  46.560001  48.549999  47.040001   26288500.0   
       2021-01-13  48.389999  47.020000  48.090000  47.220001   19097200.0   
       2021-01-14  47.660000  45.430000  47.660000  45.790001   29748700.0   
       2021-01-15  46.200001  45.080002  46.099998  45.180000   25688200.0   

                   Adj Close  
Ticker Date                   
AAPL   2009-12-31   6.492369  
       2010-01-04   6.593423  
       2010-01-05   6.604822  
       2010-01-06   6.499765  
       2010-01-07   6.487749  
...                      ...  
TWTR   2021-01-11  48.180000  
       2021-01-12  47.040001  
       2021-01-13  47.220001  
       2021-01-14  45.790001  
       2021-01-15  45.180000  

[23450 rows x 6 columns]

In [120]:
# 멀티 인덱스에서 특정 Ticker 가져오기
all_data.xs('NFLX')

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-12-31,8.011429,7.864286,7.947143,7.870000,5291300.0,7.870000
2010-01-04,7.961429,7.565714,7.931429,7.640000,17239600.0,7.640000
2010-01-05,7.657143,7.258571,7.652857,7.358571,23753100.0,7.358571
2010-01-06,7.672857,7.197143,7.361429,7.617143,23290400.0,7.617143
2010-01-07,7.757143,7.462857,7.731429,7.485714,9955400.0,7.485714
...,...,...,...,...,...,...
2021-01-11,510.730011,497.950012,507.839996,499.100006,3812700.0,499.100006
2021-01-12,501.089996,485.670013,500.000000,494.250000,5990400.0,494.250000
2021-01-13,512.349976,493.010010,495.500000,507.790009,5032100.0,507.790009


In [130]:
# 모든 포트폴리오 주식 데이터에서 조정 종가를 가져와서 인덱스처리 한다. 
adj_close = all_data[['Adj Close']].reset_index()
adj_close

,Ticker,Date,Adj Close
0,AAPL,2009-12-31,6.492369
1,AAPL,2010-01-04,6.593423
2,AAPL,2010-01-05,6.604822
3,AAPL,2010-01-06,6.499765
4,AAPL,2010-01-07,6.487749
...,...,...,...
23445,TWTR,2021-01-11,48.180000
23446,TWTR,2021-01-12,47.040001
23447,TWTR,2021-01-13,47.220001
23448,TWTR,2021-01-14,45.790001


In [148]:
# 작년 마지막 영업일과 같은 조정 종가를 가져와서 adj_close_start 에 저장 
adj_close_start = adj_close[adj_close['Date']==end_of_last_year]
adj_close_start

,Ticker,Date,Adj Close
2769,AAPL,2020-12-31,132.690002
5549,JNJ,2020-12-31,157.380005
8329,MCD,2020-12-31,214.580002
11109,MTCH,2020-12-31,151.190002
13889,NFLX,2020-12-31,540.729980
16669,GOOG,2020-12-31,1751.880005
19449,WMT,2020-12-31,144.149994
21629,FB,2020-12-31,273.160004
23439,TWTR,2020-12-31,54.150002


In [133]:
# Grab the latest stock close price
# 최종 종가를 를 가져와서 adj_close_latest에 저장 
adj_close_latest = adj_close[adj_close['Date']==stocks_end]
adj_close_latest

,Ticker,Date,Adj Close
2779,AAPL,2021-01-15,127.139999
5559,JNJ,2021-01-15,160.300003
8339,MCD,2021-01-15,209.910004
11119,MTCH,2021-01-15,150.850006
13899,NFLX,2021-01-15,497.980011
16679,GOOG,2021-01-15,1736.189941
19459,WMT,2021-01-15,144.639999
21639,FB,2021-01-15,251.360001
23449,TWTR,2021-01-15,45.180000


In [134]:
# 인덱스를 ticker로 
adj_close_latest.set_index('Ticker', inplace=True)
adj_close_latest

,Date,Adj Close
Ticker,,
AAPL,2021-01-15,127.139999
JNJ,2021-01-15,160.300003
MCD,2021-01-15,209.910004
MTCH,2021-01-15,150.850006
NFLX,2021-01-15,497.980011
GOOG,2021-01-15,1736.189941
WMT,2021-01-15,144.639999
FB,2021-01-15,251.360001
TWTR,2021-01-15,45.180000


In [135]:
# 포트폴리오 셋팅 된 데이터를 가져온다.
# portfolio_df = pd.read_excel('portfolio.xlsx')
portfolio_df = pd.read_excel('Sample stocks acquisition dates_costs.xlsx')
del portfolio_df['Unnamed: 6']
del portfolio_df['Unnamed: 7']
del portfolio_df['Unnamed: 8']
del portfolio_df['Unnamed: 9']
del portfolio_df['Unnamed: 10']

portfolio_df.head()

,Acquisition Date,Ticker,Quantity,Unit Cost,Cost Basis,Start of Year
0,2013-02-07,AAPL,125.0,65.40,8175.00,2017-12-29
1,2014-02-27,JNJ,100.0,81.90,8190.00,2017-12-29
2,2013-06-04,MCD,100.0,84.99,8499.00,2017-12-29
3,2015-12-14,MTCH,600.0,13.63,8178.00,2017-12-29
4,2016-01-14,NFLX,75.0,108.71,8153.25,2017-12-29


In [136]:
# 인덱스를 ticker 로
portfolio_df.set_index('Ticker', inplace=True)
portfolio_df

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year
Ticker,,,,,
AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29
JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29
MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29
MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29
NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29
WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29
FB,2013-12-13,150.0,53.32,7998.00,2017-12-29
TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29
NaN,NaT,NaN,NaN,NaN,NaT


In [137]:
# Merge the portfolio dataframe with the adj close dataframe; they are being joined by their indexes.
# 포트폴리오 데이터와 최종 조정 종가를 합친다. 
merged_portfolio = pd.merge(portfolio_df, adj_close_latest, left_index=True, right_index=True)
merged_portfolio

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close
Ticker,,,,,,,
AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999
JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003
MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004
MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006
NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011
WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999
FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001
TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000


In [138]:
# ticker별 수익율 컬럼(열)을 추가한다. 각 ticker별 최종 조정된 종가를 구매시 1주당 가격으로 나누고 -1을 해서 수익율을 구한 다음 'ticker return' 열에 할당한다. 
merged_portfolio['ticker return'] = merged_portfolio['Adj Close'] / merged_portfolio['Unit Cost'] -1 
merged_portfolio

,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close,ticker return
Ticker,,,,,,,,
AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037
JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265
MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820
MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499
NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811
WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716
FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179
TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891


In [149]:
merged_portfolio.reset_index(inplace=True) # 인덱스를 리셋한다.
merged_portfolio

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close,ticker return
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891


In [150]:
#sp500 데이터에서 날짜와 조정 종가를 빼서 데이터 프레임을 구성한다. 

sp_500_adj_close = sp500['Adj Close']
sp_500_adj_close 


Date
2009-12-31    1115.099976
2010-01-04    1132.989990
2010-01-05    1136.520020
2010-01-06    1137.140015
2010-01-07    1141.689941
                 ...     
2021-01-11    3799.610107
2021-01-12    3801.189941
2021-01-13    3809.840088
2021-01-14    3795.540039
2021-01-15    3768.250000
Name: Adj Close, Length: 2780, dtype: float64

In [151]:
sp500.info()
sp500.index


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2780 entries, 2009-12-31 to 2021-01-15
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   High       2780 non-null   float64
 1   Low        2780 non-null   float64
 2   Open       2780 non-null   float64
 3   Close      2780 non-null   float64
 4   Volume     2780 non-null   int64  
 5   Adj Close  2780 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 216.6 KB


DatetimeIndex(['2009-12-31', '2010-01-04', '2010-01-05', '2010-01-06',
               '2010-01-07', '2010-01-08', '2010-01-11', '2010-01-12',
               '2010-01-13', '2010-01-14',
               ...
               '2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07',
               '2021-01-08', '2021-01-11', '2021-01-12', '2021-01-13',
               '2021-01-14', '2021-01-15'],
              dtype='datetime64[ns]', name='Date', length=2780, freq=None)

In [152]:
sp500['2015-12-14':'2015-12-14']

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2015-12-14,2022.920044,1993.26001,2013.369995,2021.939941,4612440000,2021.939941


In [154]:
#sp500 데이터에서 포트폴리오 주식 취득 날짜와 같은 종가를 추가한다.
merged_portfolio_sp = pd.merge(merged_portfolio, sp_500_adj_close, left_on='Acquisition Date', right_on='Date')
merged_portfolio_sp

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Date,Adj Close_x,ticker return,Adj Close_y
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956


In [155]:
# We will delete the additional date column which is created from this merge.
# We then rename columns to Latest Date and then reflect Ticker Adj Close and SP 500 Initial Close.

# def merged_portfolio_sp['Date_y']

In [156]:
merged_portfolio_sp.rename(columns={'Date': 'Latest Date', 'Adj Close_x': 'Ticker Adj Close'
                                    , 'Adj Close_y': 'SP 500 Initial Close'}, inplace=True)

In [157]:
merged_portfolio_sp

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956


In [158]:
# This new column determines what SP 500 equivalent purchase would have been at purchase date of stock.
merged_portfolio_sp['Equiv SP Shares'] = merged_portfolio_sp['Cost Basis'] / merged_portfolio_sp['SP 500 Initial Close']
merged_portfolio_sp.head()

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419


In [159]:
# We are joining the developing dataframe with the sp500 closes again, this time with the latest close for SP.
merged_portfolio_sp_latest = pd.merge(merged_portfolio_sp, sp_500_adj_close, left_on='Latest Date', right_on='Date')

In [160]:
merged_portfolio_sp_latest

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,Adj Close
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25


In [161]:
# Once again need to delete the new Date column added as it's redundant to Latest Date.  
# Modify Adj Close from the sp dataframe to distinguish it by calling it the SP 500 Latest Close.
# del merged_portfolio_sp_latest['Date']

sp_500_adj_close

Date
2009-12-31    1115.099976
2010-01-04    1132.989990
2010-01-05    1136.520020
2010-01-06    1137.140015
2010-01-07    1141.689941
                 ...     
2021-01-11    3799.610107
2021-01-12    3801.189941
2021-01-13    3809.840088
2021-01-14    3795.540039
2021-01-15    3768.250000
Name: Adj Close, Length: 2780, dtype: float64

In [162]:
merged_portfolio_sp_latest.rename(columns={'Adj Close': 'SP 500 Latest Close'}, inplace=True)

In [163]:
merged_portfolio_sp_latest

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25


In [164]:
# Percent return of SP from acquisition date of position through latest trading day.
merged_portfolio_sp_latest['SP Return'] = merged_portfolio_sp_latest['SP 500 Latest Close'] / merged_portfolio_sp_latest['SP 500 Initial Close'] - 1


In [165]:
# This is a new column which takes the tickers return and subtracts the sp 500 equivalent range return.
merged_portfolio_sp_latest['Abs. Return Compare'] = merged_portfolio_sp_latest['ticker return'] - merged_portfolio_sp_latest['SP Return']

In [166]:
# This is a new column where we calculate the ticker's share value by multiplying the original quantity by the latest close.
merged_portfolio_sp_latest['Ticker Share Value'] = merged_portfolio_sp_latest['Quantity'] * merged_portfolio_sp_latest['Ticker Adj Close']


In [167]:
# We calculate the equivalent SP 500 Value if we take the original SP shares * the latest SP 500 share price.
merged_portfolio_sp_latest['SP 500 Value'] = merged_portfolio_sp_latest['Equiv SP Shares'] * merged_portfolio_sp_latest['SP 500 Latest Close']

In [168]:
# This is a new column where we take the current market value for the shares and subtract the SP 500 value.
merged_portfolio_sp_latest['Abs Value Compare'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['SP 500 Value']

In [169]:
# This column calculates profit / loss for stock position.
merged_portfolio_sp_latest['Stock Gain / (Loss)'] = merged_portfolio_sp_latest['Ticker Share Value'] - merged_portfolio_sp_latest['Cost Basis']

In [170]:
# This column calculates profit / loss for SP 500.
merged_portfolio_sp_latest['SP 500 Gain / (Loss)'] = merged_portfolio_sp_latest['SP 500 Value'] - merged_portfolio_sp_latest['Cost Basis']

In [171]:
merged_portfolio_sp_latest

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss)
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25,1.496538,-0.552502,15892.499924,20409.200704,-4516.700781,7717.499924,12234.200704
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25,1.032179,-0.074914,16030.000305,16643.549202,-613.548897,7840.000305,8453.549202
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25,1.309854,0.159966,20991.000366,19631.451075,1359.549292,12492.000366,11132.451075
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25,0.863680,9.203818,90510.003662,15241.178963,75268.824699,82332.003662,7063.178963
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25,0.960751,2.620060,37348.500824,15986.494640,21362.006184,29195.250824,7833.244640
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25,1.235833,-0.118117,18079.999924,19088.421134,-1008.421211,9542.499924,10550.921134
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25,1.122575,2.591603,37704.000092,16976.356044,20727.644048,29706.000092,8978.356044
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307


In [172]:
# Merge the overall dataframe with the adj close start of year dataframe for YTD tracking of tickers.
merged_portfolio_sp_latest_YTD = pd.merge(merged_portfolio_sp_latest, adj_close_start, on='Ticker')
# , how='outer'

merged_portfolio_sp_latest_YTD

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Date,Adj Close
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25,1.496538,-0.552502,15892.499924,20409.200704,-4516.700781,7717.499924,12234.200704,2020-12-31,132.690002
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25,1.032179,-0.074914,16030.000305,16643.549202,-613.548897,7840.000305,8453.549202,2020-12-31,157.380005
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25,1.309854,0.159966,20991.000366,19631.451075,1359.549292,12492.000366,11132.451075,2020-12-31,214.580002
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25,0.863680,9.203818,90510.003662,15241.178963,75268.824699,82332.003662,7063.178963,2020-12-31,151.190002
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25,0.960751,2.620060,37348.500824,15986.494640,21362.006184,29195.250824,7833.244640,2020-12-31,540.729980
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25,1.235833,-0.118117,18079.999924,19088.421134,-1008.421211,9542.499924,10550.921134,2020-12-31,144.149994
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25,1.122575,2.591603,37704.000092,16976.356044,20727.644048,29706.000092,8978.356044,2020-12-31,273.160004
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,2020-12-31,54.150002


In [173]:
# Deleting date again as it's an unnecessary column.  Explaining that new column is the Ticker Start of Year Close.
del merged_portfolio_sp_latest_YTD['Date']

In [174]:
merged_portfolio_sp_latest_YTD

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Adj Close
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25,1.496538,-0.552502,15892.499924,20409.200704,-4516.700781,7717.499924,12234.200704,132.690002
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25,1.032179,-0.074914,16030.000305,16643.549202,-613.548897,7840.000305,8453.549202,157.380005
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25,1.309854,0.159966,20991.000366,19631.451075,1359.549292,12492.000366,11132.451075,214.580002
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25,0.863680,9.203818,90510.003662,15241.178963,75268.824699,82332.003662,7063.178963,151.190002
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25,0.960751,2.620060,37348.500824,15986.494640,21362.006184,29195.250824,7833.244640,540.729980
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25,1.235833,-0.118117,18079.999924,19088.421134,-1008.421211,9542.499924,10550.921134,144.149994
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25,1.122575,2.591603,37704.000092,16976.356044,20727.644048,29706.000092,8978.356044,273.160004
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,54.150002


In [175]:
merged_portfolio_sp_latest_YTD.rename(columns={'Adj Close': 'Ticker Start Year Close'}, inplace=True)

In [176]:
merged_portfolio_sp_latest_YTD

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25,1.496538,-0.552502,15892.499924,20409.200704,-4516.700781,7717.499924,12234.200704,132.690002
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25,1.032179,-0.074914,16030.000305,16643.549202,-613.548897,7840.000305,8453.549202,157.380005
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25,1.309854,0.159966,20991.000366,19631.451075,1359.549292,12492.000366,11132.451075,214.580002
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25,0.863680,9.203818,90510.003662,15241.178963,75268.824699,82332.003662,7063.178963,151.190002
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25,0.960751,2.620060,37348.500824,15986.494640,21362.006184,29195.250824,7833.244640,540.729980
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25,1.235833,-0.118117,18079.999924,19088.421134,-1008.421211,9542.499924,10550.921134,144.149994
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25,1.122575,2.591603,37704.000092,16976.356044,20727.644048,29706.000092,8978.356044,273.160004
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,54.150002


In [177]:
sp_500_adj_close_start = sp500['Adj Close']
sp_500_adj_close_start

Date
2009-12-31    1115.099976
2010-01-04    1132.989990
2010-01-05    1136.520020
2010-01-06    1137.140015
2010-01-07    1141.689941
                 ...     
2021-01-11    3799.610107
2021-01-12    3801.189941
2021-01-13    3809.840088
2021-01-14    3795.540039
2021-01-15    3768.250000
Name: Adj Close, Length: 2780, dtype: float64

In [179]:

# Join the SP 500 start of year with current dataframe for SP 500 ytd comparisons to tickers.
merged_portfolio_sp_latest_YTD_sp = pd.merge(merged_portfolio_sp_latest_YTD, sp_500_adj_close_start
                                             , left_on='Start of Year', right_on='Date')

In [44]:
merged_portfolio_sp_latest_YTD_sp

,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start Of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,...,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,Adj Close
0,AAPL,2013-02-07,125.0,65.40,8175.00,2020-12-29,2021-01-15,127.139999,0.944037,1509.390015,...,3768.25,1.496538,-0.552502,15892.499924,20409.200704,-4516.700781,7717.499924,12234.200704,132.690002,3727.040039
1,JNJ,2014-02-27,100.0,91.90,9190.00,2020-12-29,2021-01-15,160.300003,0.744287,1854.290039,...,3768.25,1.032179,-0.287892,16030.000305,18675.728592,-2645.728287,6840.000305,9485.728592,157.380005,3727.040039
2,MCD,2013-06-04,100.0,84.99,8499.00,2020-12-29,2021-01-15,209.910004,1.469820,1631.380005,...,3768.25,1.309854,0.159966,20991.000366,19631.451075,1359.549292,12492.000366,11132.451075,214.580002,3727.040039
3,MTCH,2015-12-14,600.0,13.63,8178.00,2020-12-29,2021-01-15,150.850006,10.067499,2021.939941,...,3768.25,0.863680,9.203818,90510.003662,15241.178963,75268.824699,82332.003662,7063.178963,151.190002,3727.040039
4,NFLX,2016-01-14,75.0,108.71,8153.25,2020-12-29,2021-01-15,497.980011,3.580811,1921.839966,...,3768.25,0.960751,2.620060,37348.500824,15986.494640,21362.006184,29195.250824,7833.244640,540.729980,3727.040039


In [180]:
# Deleting another unneeded Date column.
# del merged_portfolio_sp_latest_YTD_sp['Date']

In [181]:
# Renaming so that it's clear this column is SP 500 start of year close.
merged_portfolio_sp_latest_YTD_sp.rename(columns={'Adj Close': 'SP Start Year Close'}, inplace=True)

In [182]:
merged_portfolio_sp_latest_YTD_sp

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25,1.496538,-0.552502,15892.499924,20409.200704,-4516.700781,7717.499924,12234.200704,132.690002,2673.610107
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25,1.032179,-0.074914,16030.000305,16643.549202,-613.548897,7840.000305,8453.549202,157.380005,2673.610107
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25,1.309854,0.159966,20991.000366,19631.451075,1359.549292,12492.000366,11132.451075,214.580002,2673.610107
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25,0.863680,9.203818,90510.003662,15241.178963,75268.824699,82332.003662,7063.178963,151.190002,2673.610107
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25,0.960751,2.620060,37348.500824,15986.494640,21362.006184,29195.250824,7833.244640,540.729980,2673.610107
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25,1.235833,-0.118117,18079.999924,19088.421134,-1008.421211,9542.499924,10550.921134,144.149994,2673.610107
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25,1.122575,2.591603,37704.000092,16976.356044,20727.644048,29706.000092,8978.356044,273.160004,2673.610107
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,54.150002,2673.610107


In [183]:
# YTD return for portfolio position.
merged_portfolio_sp_latest_YTD_sp['Share YTD'] = merged_portfolio_sp_latest_YTD_sp['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp['Ticker Start Year Close'] - 1

In [184]:
# YTD return for SP to run compares.
merged_portfolio_sp_latest_YTD_sp['SP 500 YTD'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Latest Close'] / merged_portfolio_sp_latest_YTD_sp['SP Start Year Close'] - 1

In [185]:
merged_portfolio_sp_latest_YTD_sp

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25,1.496538,-0.552502,15892.499924,20409.200704,-4516.700781,7717.499924,12234.200704,132.690002,2673.610107,-0.041827,0.409424
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25,1.032179,-0.074914,16030.000305,16643.549202,-613.548897,7840.000305,8453.549202,157.380005,2673.610107,0.018554,0.409424
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25,1.309854,0.159966,20991.000366,19631.451075,1359.549292,12492.000366,11132.451075,214.580002,2673.610107,-0.021763,0.409424
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25,0.863680,9.203818,90510.003662,15241.178963,75268.824699,82332.003662,7063.178963,151.190002,2673.610107,-0.002249,0.409424
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25,0.960751,2.620060,37348.500824,15986.494640,21362.006184,29195.250824,7833.244640,540.729980,2673.610107,-0.079060,0.409424
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25,1.235833,-0.118117,18079.999924,19088.421134,-1008.421211,9542.499924,10550.921134,144.149994,2673.610107,0.003399,0.409424
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25,1.122575,2.591603,37704.000092,16976.356044,20727.644048,29706.000092,8978.356044,273.160004,2673.610107,-0.079807,0.409424
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,54.150002,2673.610107,-0.165651,0.409424


In [186]:
#Investment ROI 

merged_portfolio_sp_latest_YTD_sp = merged_portfolio_sp_latest_YTD_sp.sort_values(by='Ticker', ascending=True)

In [187]:
# Cumulative sum of original investment
merged_portfolio_sp_latest_YTD_sp['Cum Invst'] = merged_portfolio_sp_latest_YTD_sp['Cost Basis'].cumsum()

In [188]:
# Cumulative sum of Ticker Share Value (latest FMV based on initial quantity purchased).
merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] = merged_portfolio_sp_latest_YTD_sp['Ticker Share Value'].cumsum()


In [189]:
# Cumulative sum of SP Share Value (latest FMV driven off of initial SP equiv purchase).
merged_portfolio_sp_latest_YTD_sp['Cum SP Returns'] = merged_portfolio_sp_latest_YTD_sp['SP 500 Value'].cumsum()


In [190]:
# Cumulative CoC multiple return for stock investments
merged_portfolio_sp_latest_YTD_sp['Cum Ticker ROI Mult'] = merged_portfolio_sp_latest_YTD_sp['Cum Ticker Returns'] / merged_portfolio_sp_latest_YTD_sp['Cum Invst']

In [191]:
merged_portfolio_sp_latest_YTD_sp

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25,1.496538,-0.552502,15892.499924,20409.200704,-4516.700781,7717.499924,12234.200704,132.690002,2673.610107,-0.041827,0.409424,8175.00,15892.499924,20409.200704,1.944037
6,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25,1.122575,2.591603,37704.000092,16976.356044,20727.644048,29706.000092,8978.356044,273.160004,2673.610107,-0.079807,0.409424,16173.00,53596.500015,37385.556748,3.313949
1,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25,1.032179,-0.074914,16030.000305,16643.549202,-613.548897,7840.000305,8453.549202,157.380005,2673.610107,0.018554,0.409424,24363.00,69626.500320,54029.105950,2.857879
2,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25,1.309854,0.159966,20991.000366,19631.451075,1359.549292,12492.000366,11132.451075,214.580002,2673.610107,-0.021763,0.409424,32862.00,90617.500687,73660.557025,2.757516
3,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25,0.863680,9.203818,90510.003662,15241.178963,75268.824699,82332.003662,7063.178963,151.190002,2673.610107,-0.002249,0.409424,41040.00,181127.504349,88901.735988,4.413438
4,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25,0.960751,2.620060,37348.500824,15986.494640,21362.006184,29195.250824,7833.244640,540.729980,2673.610107,-0.079060,0.409424,49193.25,218476.005173,104888.230628,4.441179
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,54.150002,2673.610107,-0.165651,0.409424,57378.75,228641.505241,120153.654935,3.984777
5,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25,1.235833,-0.118117,18079.999924,19088.421134,-1008.421211,9542.499924,10550.921134,144.149994,2673.610107,0.003399,0.409424,65916.25,246721.505165,139242.076069,3.742954


In [196]:
# Need to factor in that some positions were purchased much more recently than others.
# Join adj_close dataframe with portfolio in order to have acquisition date.
portfolio_df.reset_index(inplace=True)
adj_close_acq_date = pd.merge(adj_close, portfolio_df, on='Ticker')


In [197]:
# delete_columns = ['Quantity', 'Unit Cost', 'Cost Basis', 'Start Of Year']
del adj_close_acq_date['Quantity']
del adj_close_acq_date['Unit Cost']
del adj_close_acq_date['Cost Basis']
del adj_close_acq_date['Start of Year']

In [198]:
adj_close_acq_date

,Ticker,Date,Adj Close,index,Acquisition Date
0,AAPL,2009-12-31,6.492369,0,2013-02-07
1,AAPL,2010-01-04,6.593423,0,2013-02-07
2,AAPL,2010-01-05,6.604822,0,2013-02-07
3,AAPL,2010-01-06,6.499765,0,2013-02-07
4,AAPL,2010-01-07,6.487749,0,2013-02-07
...,...,...,...,...,...
20665,TWTR,2021-01-11,48.180000,7,2015-01-05
20666,TWTR,2021-01-12,47.040001,7,2015-01-05
20667,TWTR,2021-01-13,47.220001,7,2015-01-05
20668,TWTR,2021-01-14,45.790001,7,2015-01-05


In [199]:
# Sort by these columns in this order in order to make it clearer where compare for each position should begin.
adj_close_acq_date.sort_values(by=['Ticker', 'Acquisition Date', 'Date'], ascending=[True, True, True], inplace=True)


In [200]:
# Anything less than 0 means that the stock close was prior to acquisition.
adj_close_acq_date['Date Delta'] = adj_close_acq_date['Date'] - adj_close_acq_date['Acquisition Date']
adj_close_acq_date['Date Delta'] = adj_close_acq_date[['Date Delta']].apply(pd.to_numeric)

In [201]:
# Modified the dataframe being evaluated to look at highest close which occurred after Acquisition Date (aka, not prior to purchase).
adj_close_acq_date_modified = adj_close_acq_date[adj_close_acq_date['Date Delta']>=0]

In [202]:
# This pivot table will index on the Ticker and Acquisition Date, and find the max adjusted close.
adj_close_pivot = adj_close_acq_date_modified.pivot_table(index=['Ticker', 'Acquisition Date'], values='Adj Close', aggfunc=np.max)
adj_close_pivot.reset_index(inplace=True)

In [203]:
# Merge the adj close pivot table with the adj_close table in order to grab the date of the Adj Close High (good to know).
adj_close_pivot_merged = pd.merge(adj_close_pivot, adj_close
                                             , on=['Ticker', 'Adj Close'])


In [204]:
# Merge the Adj Close pivot table with the master dataframe to have the closing high since you have owned the stock.
merged_portfolio_sp_latest_YTD_sp_closing_high = pd.merge(merged_portfolio_sp_latest_YTD_sp, adj_close_pivot_merged
                                             , on=['Ticker', 'Acquisition Date'])


In [205]:
# Renaming so that it's clear that the new columns are closing high and closing high date.
merged_portfolio_sp_latest_YTD_sp_closing_high.rename(columns={'Adj Close': 'Closing High Adj Close', 'Date': 'Closing High Adj Close Date'}, inplace=True)
merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'] = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker Adj Close'] / merged_portfolio_sp_latest_YTD_sp_closing_high['Closing High Adj Close'] - 1 

In [206]:
merged_portfolio_sp_latest_YTD_sp_closing_high

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Closing High Adj Close,Closing High Adj Close Date,Pct off High
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25,1.496538,-0.552502,15892.499924,20409.200704,-4516.700781,7717.499924,12234.200704,132.690002,2673.610107,-0.041827,0.409424,8175.00,15892.499924,20409.200704,1.944037,136.690002,2020-12-28,-0.069866
1,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25,1.122575,2.591603,37704.000092,16976.356044,20727.644048,29706.000092,8978.356044,273.160004,2673.610107,-0.079807,0.409424,16173.00,53596.500015,37385.556748,3.313949,303.910004,2020-08-26,-0.172913
2,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25,1.032179,-0.074914,16030.000305,16643.549202,-613.548897,7840.000305,8453.549202,157.380005,2673.610107,0.018554,0.409424,24363.00,69626.500320,54029.105950,2.857879,160.649994,2021-01-14,-0.002179
3,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25,1.309854,0.159966,20991.000366,19631.451075,1359.549292,12492.000366,11132.451075,214.580002,2673.610107,-0.021763,0.409424,32862.00,90617.500687,73660.557025,2.757516,228.283173,2020-10-15,-0.080484
4,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25,0.863680,9.203818,90510.003662,15241.178963,75268.824699,82332.003662,7063.178963,151.190002,2673.610107,-0.002249,0.409424,41040.00,181127.504349,88901.735988,4.413438,156.119995,2020-12-22,-0.033756
5,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25,0.960751,2.620060,37348.500824,15986.494640,21362.006184,29195.250824,7833.244640,540.729980,2673.610107,-0.079060,0.409424,49193.25,218476.005173,104888.230628,4.441179,556.549988,2020-09-01,-0.105238
6,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,54.150002,2673.610107,-0.165651,0.409424,57378.75,228641.505241,120153.654935,3.984777,55.869999,2014-02-26,-0.191337
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,54.150002,2673.610107,-0.165651,0.409424,57378.75,228641.505241,120153.654935,3.984777,55.869999,2020-12-18,-0.191337
8,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25,1.235833,-0.118117,18079.999924,19088.421134,-1008.421211,9542.499924,10550.921134,144.149994,2673.610107,0.003399,0.409424,65916.25,246721.505165,139242.076069,3.742954,152.233536,2020-11-30,-0.049881


In [215]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['ticker return'][0:10],
    name = 'Ticker Total Return')
trace2 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP Return'][0:10],
    name = 'SP500 Total Return')
    
data = [trace1, trace2]
layout = go.Layout(title = 'Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [210]:
pd.set_option('display.max_columns', None)
merged_portfolio_sp_latest_YTD_sp_closing_high

,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Closing High Adj Close,Closing High Adj Close Date,Pct off High
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25,1.496538,-0.552502,15892.499924,20409.200704,-4516.700781,7717.499924,12234.200704,132.690002,2673.610107,-0.041827,0.409424,8175.00,15892.499924,20409.200704,1.944037,136.690002,2020-12-28,-0.069866
1,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25,1.122575,2.591603,37704.000092,16976.356044,20727.644048,29706.000092,8978.356044,273.160004,2673.610107,-0.079807,0.409424,16173.00,53596.500015,37385.556748,3.313949,303.910004,2020-08-26,-0.172913
2,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25,1.032179,-0.074914,16030.000305,16643.549202,-613.548897,7840.000305,8453.549202,157.380005,2673.610107,0.018554,0.409424,24363.00,69626.500320,54029.105950,2.857879,160.649994,2021-01-14,-0.002179
3,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25,1.309854,0.159966,20991.000366,19631.451075,1359.549292,12492.000366,11132.451075,214.580002,2673.610107,-0.021763,0.409424,32862.00,90617.500687,73660.557025,2.757516,228.283173,2020-10-15,-0.080484
4,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25,0.863680,9.203818,90510.003662,15241.178963,75268.824699,82332.003662,7063.178963,151.190002,2673.610107,-0.002249,0.409424,41040.00,181127.504349,88901.735988,4.413438,156.119995,2020-12-22,-0.033756
5,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25,0.960751,2.620060,37348.500824,15986.494640,21362.006184,29195.250824,7833.244640,540.729980,2673.610107,-0.079060,0.409424,49193.25,218476.005173,104888.230628,4.441179,556.549988,2020-09-01,-0.105238
6,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,54.150002,2673.610107,-0.165651,0.409424,57378.75,228641.505241,120153.654935,3.984777,55.869999,2014-02-26,-0.191337
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,54.150002,2673.610107,-0.165651,0.409424,57378.75,228641.505241,120153.654935,3.984777,55.869999,2020-12-18,-0.191337
8,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25,1.235833,-0.118117,18079.999924,19088.421134,-1008.421211,9542.499924,10550.921134,144.149994,2673.610107,0.003399,0.409424,65916.25,246721.505165,139242.076069,3.742954,152.233536,2020-11-30,-0.049881


In [214]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Stock Gain / (Loss)'][0:10],
    name = 'Ticker Total Return')
trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['SP 500 Gain / (Loss)'][0:10],
    name = 'SP500 Total Return')
trace3 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Stock Gain / (Loss)'][0:10],
    name = 'Ticker Total Return')
    
data = [trace1, trace2, trace3]
layout = go.Layout(title = 'Total Return vs S&P 500'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [223]:
pd.set_option('display.max_columns', None)
merged_portfolio_sp_latest_YTD_sp_closing_high



,index,Ticker,Acquisition Date,Quantity,Unit Cost,Cost Basis,Start of Year,Latest Date,Ticker Adj Close,ticker return,SP 500 Initial Close,Equiv SP Shares,SP 500 Latest Close,SP Return,Abs. Return Compare,Ticker Share Value,SP 500 Value,Abs Value Compare,Stock Gain / (Loss),SP 500 Gain / (Loss),Ticker Start Year Close,SP Start Year Close,Share YTD,SP 500 YTD,Cum Invst,Cum Ticker Returns,Cum SP Returns,Cum Ticker ROI Mult,Closing High Adj Close,Closing High Adj Close Date,Pct off High
0,0,AAPL,2013-02-07,125.0,65.40,8175.00,2017-12-29,2021-01-15,127.139999,0.944037,1509.390015,5.416095,3768.25,1.496538,-0.552502,15892.499924,20409.200704,-4516.700781,7717.499924,12234.200704,132.690002,2673.610107,-0.041827,0.409424,8175.00,15892.499924,20409.200704,1.944037,136.690002,2020-12-28,-0.069866
1,6,FB,2013-12-13,150.0,53.32,7998.00,2017-12-29,2021-01-15,251.360001,3.714179,1775.319946,4.505103,3768.25,1.122575,2.591603,37704.000092,16976.356044,20727.644048,29706.000092,8978.356044,273.160004,2673.610107,-0.079807,0.409424,16173.00,53596.500015,37385.556748,3.313949,303.910004,2020-08-26,-0.172913
2,1,JNJ,2014-02-27,100.0,81.90,8190.00,2017-12-29,2021-01-15,160.300003,0.957265,1854.290039,4.416785,3768.25,1.032179,-0.074914,16030.000305,16643.549202,-613.548897,7840.000305,8453.549202,157.380005,2673.610107,0.018554,0.409424,24363.00,69626.500320,54029.105950,2.857879,160.649994,2021-01-14,-0.002179
3,2,MCD,2013-06-04,100.0,84.99,8499.00,2017-12-29,2021-01-15,209.910004,1.469820,1631.380005,5.209700,3768.25,1.309854,0.159966,20991.000366,19631.451075,1359.549292,12492.000366,11132.451075,214.580002,2673.610107,-0.021763,0.409424,32862.00,90617.500687,73660.557025,2.757516,228.283173,2020-10-15,-0.080484
4,3,MTCH,2015-12-14,600.0,13.63,8178.00,2017-12-29,2021-01-15,150.850006,10.067499,2021.939941,4.044631,3768.25,0.863680,9.203818,90510.003662,15241.178963,75268.824699,82332.003662,7063.178963,151.190002,2673.610107,-0.002249,0.409424,41040.00,181127.504349,88901.735988,4.413438,156.119995,2020-12-22,-0.033756
5,4,NFLX,2016-01-14,75.0,108.71,8153.25,2017-12-29,2021-01-15,497.980011,3.580811,1921.839966,4.242419,3768.25,0.960751,2.620060,37348.500824,15986.494640,21362.006184,29195.250824,7833.244640,540.729980,2673.610107,-0.079060,0.409424,49193.25,218476.005173,104888.230628,4.441179,556.549988,2020-09-01,-0.105238
6,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,54.150002,2673.610107,-0.165651,0.409424,57378.75,228641.505241,120153.654935,3.984777,55.869999,2014-02-26,-0.191337
7,7,TWTR,2015-01-05,225.0,36.38,8185.50,2017-12-29,2021-01-15,45.180000,0.241891,2020.579956,4.051065,3768.25,0.864935,-0.623044,10165.500069,15265.424307,-5099.924238,1980.000069,7079.924307,54.150002,2673.610107,-0.165651,0.409424,57378.75,228641.505241,120153.654935,3.984777,55.869999,2020-12-18,-0.191337
8,5,WMT,2013-08-14,125.0,68.30,8537.50,2017-12-29,2021-01-15,144.639999,1.117716,1685.390015,5.065593,3768.25,1.235833,-0.118117,18079.999924,19088.421134,-1008.421211,9542.499924,10550.921134,144.149994,2673.610107,0.003399,0.409424,65916.25,246721.505165,139242.076069,3.742954,152.233536,2020-11-30,-0.049881


In [224]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Invst'][0:10],
    name = 'Cum Invst')
trace2 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker Returns'][0:10],
    name = 'Cum Ticker Returns')
trace3 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum SP Returns'][0:10],
    name = 'Cum SP Returns')

trace4 = go.Scatter(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Cum Ticker ROI Mult'][0:10],
    name = 'Cum Ticker ROI Mult'
)
    
data = [trace1, trace2, trace3, trace4]
layout = go.Layout(title = 'Total Cumulative Investments Over Time'
    , barmode = 'group'
    , yaxis=dict(title='Returns', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig)

In [227]:
trace1 = go.Bar(
    x = merged_portfolio_sp_latest_YTD_sp_closing_high['Ticker'][0:10],
    y = merged_portfolio_sp_latest_YTD_sp_closing_high['Pct off High'][0:10],
    name = 'Pct off High')
    
data = [trace1]
layout = go.Layout(title = 'Adj Close % off of High '
    , barmode = 'group'
    , yaxis=dict(title='Adj Close % off of High', tickformat=".2%")
    , xaxis=dict(title='Ticker')
    , legend=dict(x=.8,y=1)
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig)